# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy==2.1.0 allennlp==0.9.0
import spacy

     |████████████████████████████████| 27.7MB 112kB/s 
     |████████████████████████████████| 7.6MB 48.2MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 3.2MB 56.5MB/s 
     |████████████████████████████████| 2.1MB 48.4MB/s 
     |████████████████████████████████| 163kB 52.6MB/s 
     |████████████████████████████████| 317kB 54.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 133kB 56.1MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 5.6MB 45.4MB/s 
     |████████████████████████████████| 133kB 59.4MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 266kB 50.1MB/s 
     |████████████████████████████████| 245kB 55.7MB/s 
     |████████████████████████████████| 1.2MB 54.7MB/s 
     |████████████████████████████████| 7.4MB 50.7MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 

In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [3]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

from itertools import chain
vocab = Vocabulary.from_instances(chain(train_dataset, validation_dataset))

159377B [00:00, 36566686.10B/s]
562it [00:00, 18457.00it/s]
8366B [00:00, 2832772.04B/s]
23it [00:00, 4237.23it/s]
585it [00:00, 62850.10it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9117 ||: 100%|██████████| 281/281 [00:01<00:00, 206.38it/s]
accuracy: 0.7878, loss: 1.2057 ||: 100%|██████████| 12/12 [00:00<00:00, 488.81it/s]
accuracy: 0.8442, loss: 0.7298 ||: 100%|██████████| 281/281 [00:01<00:00, 256.54it/s]
accuracy: 0.7878, loss: 1.1862 ||: 100%|██████████| 12/12 [00:00<00:00, 392.86it/s]
accuracy: 0.8442, loss: 0.7161 ||: 100%|██████████| 281/281 [00:01<00:00, 260.71it/s]
accuracy: 0.7878, loss: 1.1740 ||: 100%|██████████| 12/12 [00:00<00:00, 444.63it/s]
accuracy: 0.8442, loss: 0.7072 ||: 100%|██████████| 281/281 [00:01<00:00, 236.69it/s]
accuracy: 0.7878, loss: 1.1768 ||: 100%|██████████| 12/12 [00:00<00:00, 509.77it/s]
accuracy: 0.8442, loss: 0.6988 ||: 100%|██████████| 281/281 [00:01<00:00, 266.67it/s]
accuracy: 0.7878, loss: 1.1745 ||: 100%|██████████| 12/12 [00:00<00:00, 400.80it/s]
accuracy: 0.8442, loss: 0.6915 ||: 100%|██████████| 281/281 [00:01<00:00, 274.03it/s]
accuracy: 0.7878, loss: 1.1618 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8775510204081632,
 'best_validation_loss': 0.3693773134339911,
 'epoch': 99,
 'peak_cpu_memory_MB': 3228.032,
 'peak_gpu_0_memory_MB': 1060,
 'training_accuracy': 0.9263849168520284,
 'training_cpu_memory_MB': 3228.032,
 'training_duration': '0:02:00.591272',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 1060,
 'training_loss': 0.19210356830377331,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8775510204081632,
 'validation_loss': 0.3693773134339911}

## Evaluation

The simple code below loops over the validation set, applying the model to each exmaple and collecting out the input token, gold-standard output, and model output. You can see from these methods how to access ground truth and model outputs for evaluation.

In [38]:
def tag_sentence(s):
  tag_ids = np.argmax(model.forward_on_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
## Show the first example
baseline_output[0]

[(With, 'O', 'O'),
 (a, 'O', 'O'),
 (wave, 'O', 'O'),
 (of, 'O', 'O'),
 (his, 'O', 'O'),
 (hand, 'O', 'O'),
 (,, 'O', 'O'),
 (Peng, 'B-PERSON', 'B-PERSON'),
 (Dehuai, 'I-PERSON', 'I-ORG'),
 (said, 'O', 'O'),
 (that, 'O', 'O'),
 (despite, 'O', 'O'),
 (being, 'O', 'O'),
 (over, 'O', 'O'),
 (100, 'B-CARDINAL', 'B-CARDINAL'),
 (regiments, 'O', 'O'),
 (,, 'O', 'O'),
 (let, 'O', 'O'),
 ('s, 'O', 'O'),
 (call, 'O', 'O'),
 (this, 'O', 'O'),
 (campaign, 'O', 'O'),
 (the, 'B-EVENT', 'B-ORG'),
 (Hundred, 'I-EVENT', 'I-EVENT'),
 (Regiments, 'I-EVENT', 'I-EVENT'),
 (Offensive, 'I-EVENT', 'I-EVENT'),
 (., 'O', 'O')]

In [39]:
pip install seqeval

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [40]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for base in tagged:
    word, tag, pred = base[0]
    if pred[0] == 'I':
      count += 1
      #print(pred)
    for i in range(1, len(base)):
      word1, tag1, pred1 = base[i-1]
      word2, tag2, pred2 = base[i]
      if pred1 == 'O':
        if pred2[0] == 'I':
          count += 1
          #print(pred1, pred2)
      else:
        if (pred2[0] == 'I') and (pred1[1:] != pred2[1:]):
          count += 1
          #print(pred1, pred2)
  return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# This is harsher than the token-level metric that the
# LSTM was trained to optimize, but it is the standard way
# of evaluating NER systems.
# Take tagger output as input

from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import recall_score
from seqeval.metrics import classification_report

def span_stats(tagged):
  y_trues = []
  y_preds = []
  for base in tagged:
    y_true = []
    y_pred = []
    for b in base:
      word, tag, pred = b
      y_true.append(tag)
      y_pred.append(pred)
    y_trues.append(y_true)
    y_preds.append(y_pred)
  precision = precision_score(y_trues, y_preds)
  recall = recall_score(y_trues, y_preds)
  f1 = f1_score(y_trues, y_preds)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
print(span_stats(baseline_output))

## You can check how many violations are made by the model output in predictor.
violations(baseline_output)

{'precision': 0.1746031746031746, 'recall': 0.2558139534883721, 'f1': 0.20754716981132076}


32

## Decoding

Now you can finally implement the simple Viterbi decoder. The `model` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the expression `model.forward_on_instance(s)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [45]:
# This code shows how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))
# create the k by k transition matrix
d = vocab.get_index_to_token_vocabulary('labels')
k = len(d)
#print(k)
trans_matrix = [[0]*k for _ in range(k)]
for i in range(k):
  for j in range(k):
    pre = d[i]
    nxt = d[j]
    if pre == 'O':
      if nxt[0] =='I':
        trans_matrix[i][j] = float("-inf")
    else:
      if nxt[0] == 'I' and pre[1:] != nxt[1:]:
        trans_matrix[i][j] = float("-inf")
#print(trans_matrix)


{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [46]:
#implement the viterbi and find the best path
nums = len(validation_dataset)
path = []
list_tag = [d[i] for i in d]
for i in range(nums):
  emission_scores = model.forward_on_instance(validation_dataset[i])['tag_logits']
  n = len(emission_scores)
  dp_table = [[0]*k for _ in range(n)]
  backtracking_table = [[0]*k for _ in range(n)]
  dp_table[0] = emission_scores[0]
  for word_index in range(1, n):
    for tag_index in range(k):
      tmp = [0]*k
      for j in range(k):
        tmp[j] = dp_table[word_index-1][j] + trans_matrix[j][tag_index]
      max_tmp = tmp[0]
      max_ind = 0
      for j in range(1, k):
        if tmp[j] > max_tmp:
          max_tmp = tmp[j]
          max_ind = j
      dp_table[word_index][tag_index] = emission_scores[word_index][tag_index] + max_tmp
      backtracking_table[word_index][tag_index] = d[max_ind]
  max_ = max(dp_table[-1])
  max_index = dp_table[-1].index(max_)
  tmp_path = [d[max_index]]
  i = n - 1
  while i > 0:
    tmp_tag = backtracking_table[i][max_index]
    tmp_path.append(tmp_tag)
    i -= 1
    max_index = list_tag.index(tmp_tag)
  path.append(tmp_path[::-1])

#path

In [47]:
#count the violations again with viterbi output
def violations_vi(path):
  count = 0
  for base in path:
    if base[0] == 'I':
      count += 1
    for i in range(1, len(base)):
      if base[i-1] == 'O':
        if base[i][0] == 'I':
          count += 1
      else:
        if (base[i][0] == 'I') and (base[i-1][1:] != base[i][1:]):
          count += 1
  return count

violations_vi(path)

0

In [48]:
#calculate the precision, recall and f1 score again with viterbi output
def span_stats_vi(tagged):
  y_trues = []
  for base in tagged:
    y_true = []
    for b in base:
      word, tag, pred = b
      y_true.append(tag)
    y_trues.append(y_true)
  precision = precision_score(y_trues, path)
  recall = recall_score(y_trues, path)
  f1 = f1_score(y_trues, path)
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
print(span_stats_vi(baseline_output))

{'precision': 0.4222222222222222, 'recall': 0.4418604651162791, 'f1': 0.4318181818181818}
